# Testing Random Forests with Foresight

There are two set of objects we'll have to comprehensive test.

| Basic                  | Foresight enabled        |
|------------------------|--------------------------|
| DecisionTreeClassifier | FSDecisionTreeClassifier |
| DecisionTreeRegressor  | FSDecisionTreeRegressor  | 
| RandomForestClassifier | FSRandomForestClassifier | 
| RandomForestRegressor  | FSRandomForestRegressor  |

**Basic** and **Foresight enabled** only differ in one place. In **Basic**, `n_features` features are randomly selected with *uniform weights* for all features, however, in **Foresight enabled**, `n_features` features are randomly selected with *mutual information* used as weights. 

All the arguments to be sent to both set of classes are the same as well. 


In [6]:
from TreeMethods import  DecisionTreeClassifier, DecisionTreeRegressor, RandomForestClassifier, RandomForestRegressor
from TreeMethods import  FSDecisionTreeRegressor, FSRandomForestClassifier, FSRandomForestRegressor

In [73]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification as makec
from sklearn.datasets import load_wine, load_iris       # classification 
from sklearn.datasets import load_boston, load_diabetes # regression 
from sklearn.metrics import classification_report, confusion_matrix

## Classification

We'll be comparing `FSRandomForestClassifier` with `RandomForestClassifier` to see if `Foresight` leads to any improvement.

In [40]:
w = load_wine()
i = load_iris()
###
wine = pd.DataFrame(w.data,columns=[w.feature_names])
wine['Target'] = pd.Series(data=w.target)
iris = pd.DataFrame(i.data,columns=[i.feature_names])
iris['Target'] = pd.Series(data=i.target)

In [46]:
rfc   = RandomForestClassifier.RandomForestClassifier(n_trees=10,max_depth=2,min_size=2,cost='gini')
fsrfc = FSRandomForestClassifier.FSRandomForestClassifier(n_trees=10,max_depth=2,min_size=2,cost='gini')

In [42]:
print wine.columns
print iris.columns

Index([u'alcohol', u'malic_acid', u'ash', u'alcalinity_of_ash', u'magnesium',
       u'total_phenols', u'flavanoids', u'nonflavanoid_phenols',
       u'proanthocyanins', u'color_intensity', u'hue',
       u'od280/od315_of_diluted_wines', u'proline', u'Target'],
      dtype='object')
Index([u'sepal length (cm)', u'sepal width (cm)', u'petal length (cm)',
       u'petal width (cm)', u'Target'],
      dtype='object')


In [47]:
rfc.fit(wine,target='Target')
fsrfc.fit(wine,target='Target')

TreeMethods/Foresight.py:118: RuntimeWarning: invalid value encountered in divide
  wgts = self.mi_features_y / np.sum(self.mi_features_y)
TreeMethods/Foresight.py:124: RuntimeWarning: invalid value encountered in greater
  if sample_size > np.sum(wgts > 0):
TreeMethods/Foresight.py:125: RuntimeWarning: invalid value encountered in greater
  sample_size = np.sum(wgts > 0)
TreeMethods/Foresight.py:133: RuntimeWarning: invalid value encountered in less
  get_a_heap_of_features = np.random.choice(self.d, size=sample_size, replace=False, p=wgts)
TreeMethods/Foresight.py:133: RuntimeWarning: invalid value encountered in greater
  get_a_heap_of_features = np.random.choice(self.d, size=sample_size, replace=False, p=wgts)
TreeMethods/Foresight.py:134: RuntimeWarning: invalid value encountered in less
  get_1_feature = np.random.choice(self.d, size=1, replace=False, p=wgts)[0]
TreeMethods/Foresight.py:134: RuntimeWarning: invalid value encountered in greater
  get_1_feature = np.random.choice(s

ValueError: Fewer non-zero entries in p than size

In [63]:
res = []
for idx in wine.index:
    res.append(rfc.predict(wine.loc[[idx]].squeeze()))

In [74]:
print confusion_matrix(res,wine['Target'])

[[57  1  0]
 [ 2 69 10]
 [ 0  1 38]]


In [71]:
tn = ['class 0', 'class 1', 'class 2']
print classification_report(res,wine['Target'],target_names=tn)

             precision    recall  f1-score   support

    class 0       0.97      0.98      0.97        58
    class 1       0.97      0.85      0.91        81
    class 2       0.79      0.97      0.87        39

avg / total       0.93      0.92      0.92       178

